<a href="https://colab.research.google.com/github/Hironobu-Kawaguchi/kmnist/blob/master/colab_kmnist_pytorch_cnn_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as dataset
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        # x = F.relu(x)   # add
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # loss = F.nll_loss(output, target)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [0]:
def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                        help='how many batches to wait before logging training status')

    parser.add_argument('--save-model', action='store_true', default=True,
                        help='For Saving the current Model')
    # args = parser.parse_args()
    args = parser.parse_args(args=[])
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.KMNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.KMNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "kmnist_cnn.pt")


In [6]:
main()

0it [00:00, ?it/s]

 99%|█████████▉| 18046976/18165135 [00:10<00:00, 4451055.52it/s]

Extracting ../data/KMNIST/raw/train-images-idx3-ubyte.gz to ../data/KMNIST/raw



0it [00:00, ?it/s]


 56%|█████▌    | 16384/29497 [00:00<00:00, 73198.02it/s]
32768it [00:00, 48183.28it/s]                           
0it [00:00, ?it/s]

...

Extracting ../data/KMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/KMNIST/raw



  0%|          | 0/5120 [00:00<?, ?it/s]
8192it [00:00, 18305.38it/s]            

Extracting ../data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/KMNIST/raw
Processing...
Done!
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.336921
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.561426
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.345984


18169856it [00:30, 4451055.52it/s]                              

Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.360342
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.425967
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.275108
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.192966
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.213575
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.210393
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.414557

Test set: Average loss: 0.3355, Accuracy: 9092/10000 (91%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.082735
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.053388
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.137213
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.314053
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.209911
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.085720
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.193942
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.085430
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.205925
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.182198

Test set: Average loss: 0.2699, Accuracy: 9354/10000 (94%)

Train Epoch: 3 [0/60000 (0%)]	Loss

In [0]:
torch.__version__

'1.3.1'

In [0]:
import torchvision
torchvision.__version__

'0.4.2'

In [0]:
np.__version__

'1.17.4'

In [0]:
import sys
sys.version

'3.6.9 (default, Nov  7 2019, 10:44:02) \n[GCC 8.3.0]'